In [159]:
from __future__ import print_function
import numpy as np
from six.moves import cPickle as pickle
from six.moves import range
import scipy.signal

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000, 10)
Validation set (10000, 28, 28) (10000, 10)
Test set (10000, 28, 28) (10000, 10)


In [77]:
def sigmoid(inX):  
    return 1.0/(1.0+np.exp(-inX))

In [389]:
def max_pooling(data,pooling):
    output = np.zeros([data.shape[0]/pooling[0],data.shape[1]/pooling[1]])
    for i in range(output.shape[0]):
        for j in range(output.shape[1]):
            output[i,j] = data[i*pooling[0]:(i+1)*pooling[0],j*pooling[1]:(j+1)*pooling[1]].mean()
    return output

def softmax(x):
    return np.exp(x-x.max(0)) / np.sum(np.exp(x-x.max(0)), axis=0)

def rot180(conv_kernel):
    return np.fliplr( np.flipud(conv_kernel))

In [149]:
class layer(object):
    w = []
    b = []
    type = ''
    input_nums = 0
    output_nums = 0
    kernel_size = 0
    mapsize = []
    a = []
    d = []
    dw = []
    db = []
    #mapsize means input size, it's a 2 nums vector
    #input_nums means what's the numbers of feature/channels
    #out_nums means whate's the numbers of output feature 
    #kernel_size means if it is convertional layer, its kernel size
    #type_size means the layer's type: 's' means pooling layer, 'c' means convertional layer, 'f' means full-connected layer
    def __init__(self,_mapsize,in_n,out_n,k_size,type_str):
        self.mapsize = _mapsize
        self.input_nums = in_n
        self.output_nums = out_n
        self.kernel_size = k_size
        self.type = type_str

In [331]:
class options(object):
    alpha = 0.1
    epochs = 5
    batch = 200

In [592]:
class cnn(object):
    cnn_layer = []
    output_d = []
    def __init__(self,x, kernel_size,y_size):
        input_size = x[0].shape
        mapsize = [input_size[0]-kernel_size+1,input_size[1]-kernel_size+1]
        self.cnn_layer.append(layer(mapsize,1,6,kernel_size,'c'))
        
        mapsize = [mapsize[0]/2,mapsize[1]/2]
        self.cnn_layer.append(layer(mapsize,6,6,2,'s'))
        
        mapsize = [mapsize[0]-5+1,mapsize[1]-5+1]
        self.cnn_layer.append(layer(mapsize,6,12,5,'c'))
        
        mapsize = [mapsize[0]/2,mapsize[1]/2]
        self.cnn_layer.append(layer(mapsize,12,12,2,'s'))
        
        mapsize = mapsize[0]*mapsize[1]*12
        self.cnn_layer.append(layer(mapsize,12,y_size,1,'f'))
    
    def init_w(self):
        self.cnn_layer[0].w = np.random.randn(self.cnn_layer[0].input_nums,self.cnn_layer[0].output_nums,self.cnn_layer[0].kernel_size,self.cnn_layer[0].kernel_size)
        self.cnn_layer[1].w = np.ones([self.cnn_layer[1].kernel_size,self.cnn_layer[1].kernel_size])/4
        self.cnn_layer[2].w = np.random.randn(self.cnn_layer[2].input_nums,self.cnn_layer[2].output_nums,self.cnn_layer[2].kernel_size,self.cnn_layer[2].kernel_size)
        self.cnn_layer[3].w = np.ones([self.cnn_layer[3].kernel_size,self.cnn_layer[1].kernel_size])/4
        self.cnn_layer[4].w = np.random.randn(self.cnn_layer[4].mapsize,self.cnn_layer[4].output_nums)
        
        self.cnn_layer[0].b = np.zeros(self.cnn_layer[0].output_nums)
        self.cnn_layer[1].b = 0
        self.cnn_layer[2].b = np.zeros(self.cnn_layer[2].output_nums)
        self.cnn_layer[3].b = 0
        self.cnn_layer[4].b = np.zeros([1,self.cnn_layer[4].output_nums])
    
    def cnnff(self,x):
        nums = x.shape[0]
        #compute 1st layer-Convolutional layer
        in_n = self.cnn_layer[0].input_nums
        out_n = self.cnn_layer[0].output_nums
        self.cnn_layer[0].a = np.zeros([nums,out_n]+self.cnn_layer[0].mapsize)
        for k in xrange(nums):
            for i in xrange(out_n):
                z = np.zeros(self.cnn_layer[0].mapsize)
                for j in xrange(in_n):
                    z += scipy.signal.convolve2d(x[k],self.cnn_layer[0].w[j,i],'valid')
                self.cnn_layer[0].a[k][i] = sigmoid(z+self.cnn_layer[0].b[i])

                   
        #compute 2nd layer-Pooling layer
        out_n = self.cnn_layer[0].output_nums
        self.cnn_layer[1].a = np.zeros([nums,out_n]+self.cnn_layer[1].mapsize)
        for k in xrange(nums):
            for i in xrange(out_n):
                self.cnn_layer[1].a[k][i] = max_pooling(self.cnn_layer[0].a[k][i],[2,2])
        

        #compute 3nd layer-Convolutional layer
        in_n = self.cnn_layer[2].input_nums
        out_n = self.cnn_layer[2].output_nums
        self.cnn_layer[2].a = np.zeros([nums,out_n]+self.cnn_layer[2].mapsize)
        for k in xrange(nums):
            for i in xrange(out_n):
                z = np.zeros(self.cnn_layer[2].mapsize)
                for j in xrange(in_n):
                    z += scipy.signal.convolve2d(self.cnn_layer[1].a[k][j],self.cnn_layer[2].w[j,i],'valid')
                self.cnn_layer[2].a[k][i] = sigmoid(z+self.cnn_layer[2].b[i])
        
        
        #compute 4th layer-Pooling layer
        out_n = self.cnn_layer[3].output_nums
        self.cnn_layer[3].a = np.zeros([nums,out_n]+self.cnn_layer[3].mapsize)
        for k in xrange(nums):
            for i in xrange(out_n):
                self.cnn_layer[3].a[k][i] = max_pooling(self.cnn_layer[2].a[k][i],[2,2])
        

        #compute 5nd layer-softmax output layer
        feature_vector = np.array(self.cnn_layer[3].a).reshape(nums,192)
        raw_output = np.dot(feature_vector,self.cnn_layer[4].w)+np.matlib.repmat(self.cnn_layer[4].b,nums,1)
        self.cnn_layer[4].a = softmax(raw_output.T).T
        #print(self.cnn_layer[0].a[0][0][0])
        
    def compute_error(self,y):
        tmp = np.log(self.cnn_layer[4].a)*y
        
        return (-tmp.sum()/y.shape[0],(np.argmax(y,1) == np.argmax(self.cnn_layer[4].a,1) ).astype(np.float32).sum()/y.shape[0])
    
    def cnnbp(self,x,y):
        nums = y.shape[0]
        #compute 5st layer-error
        #self.cnn_layer[4].d is a k*10 matrix
        self.cnn_layer[4].d = (self.cnn_layer[4].a-y)
        
        
        #compute 4th layer-error
        #tmp_d is a k*192 matrix
        tmp_d = np.dot(self.cnn_layer[4].d,self.cnn_layer[4].w.T)
        out_n = self.cnn_layer[3].output_nums
        tmp_size = self.cnn_layer[3].mapsize
        #self.cnn_layer[4].d is a k*12*4*4 matrix
        self.cnn_layer[3].d = np.zeros([nums,out_n]+self.cnn_layer[3].mapsize)
        
        for k in xrange(nums):
            for i in xrange(out_n):
                self.cnn_layer[3].d[k][i] = (tmp_d[k][i*tmp_size[0]*tmp_size[1]:(i+1)*tmp_size[0]*tmp_size[1]]).reshape(tmp_size[0],tmp_size[1])
        #print(out_n,self.cnn_layer[3].d[0][0].shape)
        
        #compute 3rd layer-error
        #self.cnn_layer[3]
        self.cnn_layer[2].d = np.zeros([nums,out_n]+self.cnn_layer[2].mapsize)
        for k in xrange(nums):
            for t in xrange(out_n):
                for i in xrange(tmp_size[0]):
                    for j in xrange(tmp_size[1]):
                        self.cnn_layer[2].d[k][t][i,j] = self.cnn_layer[3].d[k][t][i/2,j/2]
                self.cnn_layer[2].d[k][t] = self.cnn_layer[2].d[k][t]/4 * (1-self.cnn_layer[2].a[k][t])*self.cnn_layer[2].a[k][t]
        #print(out_n,self.cnn_layer[2].d[0][0].shape)
        
        #compute 2rd layer-error
        out_n = self.cnn_layer[2].output_nums
        in_n = self.cnn_layer[2].input_nums
        self.cnn_layer[1].d = np.zeros([nums,in_n]+self.cnn_layer[1].mapsize)
        for k in xrange(nums):
            for i in xrange(in_n):
                z = np.zeros(self.cnn_layer[1].mapsize)
                for j in xrange(out_n):
                    z += scipy.signal.convolve2d(self.cnn_layer[2].d[k][j],rot180(self.cnn_layer[2].w[i][j]),'full')
                self.cnn_layer[1].d[k][i] = z;
        #print(in_n,self.cnn_layer[1].d[0][0].shape)
        
        #compute 1st layer-error
        #self.cnn_layer[0]
        tmp_size = self.cnn_layer[0].mapsize
        self.cnn_layer[0].d = np.zeros([nums,in_n]+tmp_size)
        for k in xrange(nums):
            for t in xrange(in_n):
                for i in xrange(tmp_size[0]):
                    for j in xrange(tmp_size[1]):
                        self.cnn_layer[0].d[k][t][i,j] = self.cnn_layer[1].d[k][t][i/2,j/2]
                self.cnn_layer[0].d[k][t] = self.cnn_layer[0].d[k][t]/4 * (1-self.cnn_layer[0].a[k][t])*self.cnn_layer[0].a[k][t]
        #print(in_n,self.cnn_layer[0].d[0][0].shape)
        
        
        #compute grantiant of w and b:
        #compute 1st layer-conventional layer
        out_n = self.cnn_layer[0].output_nums
        in_n = self.cnn_layer[0].input_nums
        tmp_size = self.cnn_layer[0].mapsize
        self.cnn_layer[0].dw = np.zeros([in_n,out_n,self.cnn_layer[0].kernel_size,self.cnn_layer[0].kernel_size])
        self.cnn_layer[0].db = [0]*out_n
        for j in xrange(out_n):
            for i in xrange(in_n):
                for k in xrange(nums):
                    self.cnn_layer[0].dw[i][j] += scipy.signal.convolve2d(rot180(x[k]),self.cnn_layer[0].d[k][j],'valid')
                self.cnn_layer[0].dw[i][j] = self.cnn_layer[0].dw[i][j]/nums;
            for k in xrange(nums):
                self.cnn_layer[0].db[j] += self.cnn_layer[0].d[k][j].sum()
            self.cnn_layer[0].db[j] = self.cnn_layer[0].db[j]/nums;
        
        #compute 3nd layer-conventional layer
        out_n = self.cnn_layer[2].output_nums
        in_n = self.cnn_layer[2].input_nums
        tmp_size = self.cnn_layer[0].mapsize
        self.cnn_layer[2].dw = np.zeros([in_n,out_n,self.cnn_layer[2].kernel_size,self.cnn_layer[2].kernel_size])
        self.cnn_layer[2].db = [0]*out_n
        for j in xrange(out_n):
            for i in xrange(in_n):
                for k in xrange(nums):
                    self.cnn_layer[2].dw[i][j] += scipy.signal.convolve2d(rot180(self.cnn_layer[1].a[k][i]),self.cnn_layer[2].d[k][j],'valid')
                self.cnn_layer[2].dw[i][j] = self.cnn_layer[2].dw[i][j]/nums;
            for k in xrange(nums):
                self.cnn_layer[2].db[j] += self.cnn_layer[2].d[k][j].sum()
            self.cnn_layer[2].db[j] = self.cnn_layer[2].db[j]/nums;
        
        #compute the output layer:
        feature_vector = np.array(self.cnn_layer[3].a).reshape(nums,192)
        self.cnn_layer[4].dw = np.dot(feature_vector.T,self.cnn_layer[4].d)/nums
        self.cnn_layer[4].db = self.cnn_layer[4].d.mean(0)
    
    def cnnapplygrads(self,options):
        #applygrads in 1st layer:
        out_n = self.cnn_layer[0].output_nums
        in_n = self.cnn_layer[0].input_nums
        for j in xrange(out_n):
            for i in xrange(in_n):
                self.cnn_layer[0].w[i][j] = self.cnn_layer[0].w[i][j] - options.alpha*self.cnn_layer[0].dw[i][j]
            self.cnn_layer[0].b[j] = self.cnn_layer[0].b[j] - options.alpha*self.cnn_layer[0].db[j]
        
        #applygrads in 3nd layer:
        out_n = self.cnn_layer[2].output_nums
        in_n = self.cnn_layer[2].input_nums
        for j in xrange(out_n):
            for i in xrange(in_n):
                self.cnn_layer[2].w[i][j] = self.cnn_layer[2].w[i][j] - options.alpha*self.cnn_layer[2].dw[i][j]
            self.cnn_layer[2].b[j] = self.cnn_layer[2].b[j] - options.alpha*self.cnn_layer[2].db[j]
            
        #applygrads in output layer:
        self.cnn_layer[4].w = self.cnn_layer[4].w - options.alpha*self.cnn_layer[4].dw
        self.cnn_layer[4].b = self.cnn_layer[4].b - options.alpha*self.cnn_layer[4].db
        
    def train(self,train_dataset,train_labels,options):
        nums = train_dataset.shape[0]
        for i in xrange(options.epochs):
            for j in xrange(nums/options.batch):
                x_tmp = train_dataset[j*options.batch:(j+1)*options.batch]
                y_tmp = train_labels[j*options.batch:(j+1)*options.batch]
                self.cnnff(x_tmp)
                print(tmp.compute_error(y_tmp))
                self.cnnbp(x_tmp,y_tmp)
                self.cnnapplygrads(opt)
    
    def checkgrad(self,x,y):
        epsilon = 1e-2;
        res = np.zeros([1,10])
        self.cnnff(x_tmp)
        e0 = self.compute_error(y_tmp)
        
        for i in xrange(10):
            
            self.cnn_layer[4].b[0][i] + epsilon
            self.cnnff(x_tmp)
            e1 = self.compute_error(y_tmp)
            res[0][i] = (e1[0]-e0[0])/epsilon
            self.cnn_layer[4].b[0][i] - epsilon
        print( res)

In [595]:
tmp = cnn(x_tmp,5,10)
tmp.init_w()


In [579]:
def checkgrad(tmp,x,y):
    epsilon = 1e-5;
    res = np.zeros([1,10])
    tmp.cnnff(x_tmp)
    e0 = tmp.compute_error(y_tmp)
    print(e0)
    for i in xrange(10):

        tmp.cnn_layer[4].b[0][i] += epsilon
        tmp.cnnff(x_tmp)
        e1 = tmp.compute_error(y_tmp)
        #print (e1)
        res[0][i] = (e1[0]-e0[0])/epsilon
        tmp.cnn_layer[4].b[0][i] -= epsilon
    print( res)

In [593]:
opt.batch = 100

In [ ]:
tmp.train(train_dataset,train_labels,opt)


(15.231995699094218, 0.089999999999999997)
(11.163125928251588, 0.14000000000000001)
(9.1055956204853814, 0.17000000000000001)
(8.092762276910733, 0.14000000000000001)
(8.1533559334141366, 0.14999999999999999)
(6.8299446484822912, 0.14999999999999999)
(8.014883381538171, 0.080000000000000002)
(6.9797832216559241, 0.11)
(6.0099938163880147, 0.13)
(4.9285826602378915, 0.14000000000000001)
(5.5879925598018936, 0.14000000000000001)
(4.9609309667686725, 0.17000000000000001)
(4.9769203910655664, 0.070000000000000007)
(5.4472834712584666, 0.14999999999999999)
(4.6436202209710133, 0.12)
(4.4039758423988848, 0.17000000000000001)
(4.5791169539932897, 0.080000000000000002)
(3.6797195370047144, 0.12)
(4.2994115674460325, 0.11)
(3.8206554698279716, 0.23000000000000001)
(4.4699398227586631, 0.16)
(4.1300035623678832, 0.13)
(4.1131261190280766, 0.17999999999999999)
(3.7516019131574869, 0.16)
(3.7192659572122593, 0.22)
(3.9688976758458807, 0.12)
(4.0994048900444344, 0.17999999999999999)
(3.67386193167

In [580]:
checkgrad(tmp,x_tmp,y_tmp)

20 1 6
(4.2959506338792126, 0.80000000000000004)
20 1 6
20 1 6
20 1 6
20 1 6
20 1 6
20 1 6
20 1 6
20 1 6
20 1 6
20 1 6
[[-0.03668057  0.034404    0.02379609  0.02015678 -0.05       -0.05
  -0.04765788  0.          0.10598367  0.        ]]


In [582]:
tmp.cnn_layer[4].db

array([-0.05528841, -0.05580833, -0.05806376, -0.0521363 , -0.05      ,
       -0.05      , -0.05664398,  0.        , -0.05585884,  0.        ])

In [379]:
softmax(m2D)

array([[ 0.04742587,  0.00247262,  0.04742587],
       [ 0.95257413,  0.99752738,  0.95257413]])

In [206]:
np.exp(x) / np.sum(np.exp(x), axis=0)

0.0

In [363]:
tt = m2D.max(axis= 1)
np.matlib.repmat(tt.T,1,2)

array([[3, 5, 3, 5]])

In [377]:
m2D = np.array([[1,2,3],[4,8,6]])

In [404]:
(np.argmax(y_tmp,1) == np.argmax(y_tmp,1) ).sum()

200

In [402]:
tmp.cnn_layer[4].a[0:5]

array([[ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan]])